# Simple HMM in NumPy or PyTorch

In [ ]:
%run /home/hanzl/f-w/prak/prongen/hmm_pron.py

In [ ]:
%run /home/hanzl/f-w/prak/acmodel/matrix.py

In [ ]:
A = m([[0.7,0.3,0,0],
       [0,0.9,0.1,0],
       [0,0,0.8,0.2],
       [0,0,0,1]])

A = m(hmm.A)

In [ ]:
A.size()[0]

In [ ]:
# where we can be
#x_m = m([[1,0,0,0]])
x_list = [1,0,0,0]
x_list = [1]+[0]*(A.size()[0]-1)


x_m = m([x_list])
x_rev = m([list(reversed(x_list))])

In [ ]:
x_m.size()

In [ ]:
# recording length in frames
tmax = 14000

tmax = 363

tmax = int(363/35 * A.size()[0])
tmax

In [ ]:
len_x = x_m.size()[1]

# allocate space for mantissa-like (kept in range) and row-exponent values
alpha_m = m.rowlist((tmax,len_x))
alpha_exp = m.rowlist((tmax,1))

exponent = 0
base = 1000 # base for separation to exp and m (somewhere around 5e-324 starts underflow)

In [ ]:
# FAKE b VALUES:
b = m.zeros((tmax,len_x)) + 0.1

In [ ]:
#x_m = m([[1,0,0,0]]) 

In [ ]:
%%time
#x_m = m([[1,0,0,0]]) # mantissa-like moderated value (kept in range)
for row in range(tmax):
    while x_m.max()<1/base: # renormalize and remember power of base used
        x_m *= base
        exponent -= 1
    alpha_exp[row] = exponent
    alpha_m[row] = x_m
    x_m = x_m@A*b[row]   # FINETUNE WHICH row's b IS USED (ALSO FOR beta)

### Size 14000 x 4
* GPU: 7.01 torch sparse coo, NotImpl torch sparse csr, 0.384 torch dense
* CPU: 1.54 torch sparse coo, 0.210 torch sparse csr, 0.214 torch dense

### Size 363 x 35
* GPU: ??? torch sparse coo, NotImpl torch sparse csr, 0.3770 torch dense
* CPU: ??? torch sparse coo, NotImpl torch sparse csr, 0.0054 torch dense

715 15.4 CPU 391 GPU

1420 33  CPU 434 GPU

2831  78.5 CPU 486 GPU

5652  209  CPU  612 GPU

11294  529 CPU  885 GPU

22578    5000 CPU   1860 GPU

45146    120000 CPU  6110 GPU   unusable GPU coo   not impl CPU csr   unusable CPU coo


In [ ]:
# Now we will go backward
At = A.T()

In [ ]:
beta_m = m.rowlist((tmax,len_x))
beta_exp = m.rowlist((tmax,1))

exponent = 0

In [ ]:
#x_m = m([[0,0,0,1]])
x_m = x_rev
for row in range(tmax-1,0-1,-1):
    beta_m[row] = x_m
    beta_exp[row] = exponent
    x_m = x_m@At*b[row]
    while x_m.max()<1/base: # renormalize and remember power of base used
        x_m *= base
        exponent -= 1

In [ ]:
L_m = alpha_m*beta_m # this is .*

In [ ]:
L_exp = alpha_exp+beta_exp

In [ ]:
L_exp += float(-L_exp.max())

In [ ]:
# re-normalize L
for row in range(tmax):
    while float(L_exp[row].val)<0: # renormalize and remember power of base used
        L_m[row] *= 1/base  # invokes setitem which converts L_m to dense
        L_exp[row] += 1

In [ ]:
L_m.size()

In [ ]:
# test - should be the same now
L_exp.min(), L_exp.max()

In [ ]:
#L_m

In [ ]:
L_m.max()

In [ ]:
X = (L_m*(1/float(L_m.max()))).val

In [ ]:
#X

In [ ]:
X.size()

In [ ]:
scale = 1/X.max(1).values
Y = (X.T*scale).T

In [ ]:
import matplotlib.pyplot as plt
%matplotlib ipympl
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (20,3)
# Use full with and waste less space on prompts on the left:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.prompt_container{width: 11ex !important; }</style>"))
display(HTML("<style>div.prompt{min-width: 11ex; }</style>"))

In [ ]:
plt.clf()
p = plt.plot(Y*(Y>0.99))
plt.show()

In [ ]:
i = X.max(1).indices

In [ ]:
"".join([hmm.b[ii] for ii in i])

In [ ]:
hmm.wav

In [ ]:
import torchaudio
cmap = ['BrBG', 'RdBu', 'PuOr'][0]
def plot_specgram(waveform, sample_rate):
    waveform = waveform.numpy()
    figure = plt.figure()
    plt.specgram(waveform[0], Fs=sample_rate, cmap=cmap)
    plt.tick_params(axis="x",direction="in", pad=-12)
    plt.tick_params(axis="y",direction="in", pad=-38)
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
    plt.show(block=False)
    figure.canvas.header_visible = False
    figure.canvas.footer_visible = False
    #plt.clf()


In [ ]:
def plot_waveform(waveform, sample_rate):
    waveform = waveform.numpy()
    num_channels, num_frames = waveform.shape
    time_axis = torch.arange(0, num_frames) / sample_rate
    figure, axes = plt.subplots(num_channels, 1)
    #axes = plt
    axes.tick_params(axis="x",direction="in", pad=-12)
    axes.tick_params(axis="y",direction="in", pad=-22)
    axes.plot(time_axis, waveform[0], linewidth=1, color=(0,0.4,0.4))
    axes.grid(True)
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
    plt.show(block=False)
    max_time = num_frames/sample_rate
    axes.set_xlim([0,max_time])
    figure.canvas.header_visible = False
    figure.canvas.footer_visible = False
    #plt.clf()

In [ ]:
def plot_matrix(x):
    figure = plt.figure()
    plt.imshow(x.T)
    plt.tick_params(axis="x",direction="in", pad=-12)
    plt.tick_params(axis="y",direction="in", pad=-38)
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
    plt.show(block=False)
    figure.canvas.header_visible = False
    figure.canvas.footer_visible = False
    #plt.clf()

In [ ]:
plot_matrix(Y)

In [ ]:
waveform, sample_rate = torchaudio.load(hmm.wav)

In [ ]:
mfcc = torchaudio.compliance.kaldi.mfcc(waveform) # default 16kHz is correct for this file

In [ ]:
plot_matrix(mfcc)

In [ ]:
plot_specgram(waveform, sample_rate)

In [ ]:
plot_waveform(waveform, sample_rate)

In [ ]:
print(hmm)